In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df = pd.read_excel('../data/color.xlsx',engine='openpyxl')
df.head()

,X1,X2,Color
0,-2.5,2.8,Blue
1,-1.5,1.8,Blue
2,-0.8,2.8,Blue
3,-0.3,0.8,Blue
4,1.1,2.1,Blue


In [3]:
clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(df.iloc[:,:2], df.Color)
clf.predict([[0.8, -0.2]])

array(['Yellow'], dtype=object)

In [4]:
def pre(point,df):
    df_dummies = pd.get_dummies(df.Color)
    reg = KNeighborsRegressor(n_neighbors=6)
    L=[]
    for col in df_dummies.columns:
        reg.fit(df.iloc[:,:2], df_dummies[col])
        reg.predict([point])
        L.append(reg.predict([point])[0])
    return df_dummies.columns[L.index(max(L))]
pre([0.8, -0.2],df)

'Yellow'

In [5]:
df = pd.read_csv('../data/audit.csv')
df.head()

,ID,Age,Employment,Marital,Income,Gender,Hours
0,1004641,38,Private,Unmarried,81838.00,Female,72
1,1010229,35,Private,Absent,72099.00,Male,30
2,1024587,32,Private,Divorced,154676.74,Male,40
3,1038288,45,Private,Married,27743.82,Male,55
4,1044221,60,Private,Married,7568.23,Male,40


先将婚配、性别转化为数字，再添加年龄、收入和时间（注意要归一化），最后加上雇佣情况。将数据根据是否有雇佣情况分成训练集和测试集。将雇佣情况转化为数值向量。遍历每种情况，将训练集传入训练器，预判测试集，综合情况结果。寻找最可能情况，转化为原形式，再填入原数据。

In [8]:
res_df = df.copy()
fea_df=pd.concat([pd.get_dummies(df[['Marital', 'Gender']]),df[['Age','Income','Hours']].apply(lambda x:(x-x.min())/(x.max()-x.min())), df.Employment],1)
X_train = fea_df.loc[fea_df.Employment.notna(),:]
X_test = fea_df.loc[fea_df.Employment.isna(),:]
df_dummies = pd.get_dummies(X_train.Employment)
stack_list = []
for col in df_dummies.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(X_train.iloc[:,:-1], df_dummies[col])
    res = clf.predict(X_test.iloc[:,:-1]).reshape(-1,1)
    stack_list.append(res)

code_res = pd.Series(np.hstack(stack_list).argmax(1))
cat_res = code_res.replace(dict(zip(list(range(df_dummies.shape[0])),df_dummies.columns)))
res_df.loc[res_df.Employment.isna(), 'Employment'] = cat_res.values
res_df.isna().sum()

ID            0
Age           0
Employment    0
Marital       0
Income        0
Gender        0
Hours         0
dtype: int64